In [193]:
import itertools as it
from matplotlib import gridspec
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pennylane as qml
import qutip
import scipy as sp
from tqdm.auto import tqdm
import pandas as pd
from os.path import join

In [194]:
from read_data import load_data
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='ppt', choices=['train', 'valid', 'test'], help='dataset split used to decode')
parser.add_argument('--data_dir', type=str, default='/mnt/nas2/home/tangyehui/src/DiffuSeq/data/N_10_NM_1000_S_20', help='path to the folder of diffusion model')
parser.add_argument('--seed', type=int, default=101, help='random seed')

parser.add_argument('--num_measurements', type=int, default=1000, help='batch size')
parser.add_argument('--num_qubits', type=int, default=10, help='batch size')
parser.add_argument('--split', type=str, default='train', choices=['train', 'valid', 'test'], help='dataset split used to decode')
parser.add_argument('--shots', type=int, default=1000, help='number of shots for generation')
args, unknown = parser.parse_known_args()

data = load_data(args, split=args.split, shots=args.shots)


############################## 
Loading dataset ppt from /mnt/nas2/home/tangyehui/src/DiffuSeq/data/N_10_NM_1000_S_20...
### Total number of samples: 20
### The dimension of conditions: 9
### The dimension of measurements: 10
### Data samples...
 [[1.82 0.98 1.3  0.16 1.87 1.56 1.31 1.68 0.47]
 [0.36 1.53 0.18 1.6  1.54 0.17 0.41 0.06 0.17]] 
 [[5 3 4 5 0 5 3 0 4 2]
 [5 2 5 0 1 5 1 0 5 4]]


In [386]:
def load_coupling_matrix(coupling_strength):
    rows = 1
    cols = len(coupling_strength)+1

    qubits = rows * cols
    
    # Create a 2D Lattice
    edges = [
        (si, sj) for (si, sj) in it.combinations(range(qubits), 2)
        if ((sj % cols > 0) and sj - si == 1) or sj - si == cols
    ]
    
    # sample edge weights uniformly at random from [0, 2]
    edge_weights = coupling_strength

    coupling_matrix = np.zeros((qubits, qubits))
    for (i, j), w in zip(edges, edge_weights):
        coupling_matrix[i, j] = coupling_matrix[j, i] = w
        
    return coupling_matrix

# define the system size and lattice geometry
rows, cols = 1, 10
wires = rows * cols

# sample a coupling matrix
J_list = [load_coupling_matrix(J) for J in data['conditions']]
samples = pd.read_csv('generation_outputs/results/conditional_heisenberg_N10/model/ns1000/iter50000_lr0.001_wd0_bs512_dropout0.0_lrschedulewarmup_cosine20240518-014155/checkpoints/checkpoint_50000.pth.tar/samples_all.txt'
    , header=None).to_numpy(dtype=int).reshape(-1, 50000, 10)

In [387]:
def build_hamiltonian(coupling_matrix):
    coeffs, ops = [], []
    ns = coupling_matrix.shape[0]

    for i, j in it.combinations(range(ns), r=2):
        coeff = coupling_matrix[i, j]
        if coeff:
            for op in [qml.PauliX, qml.PauliY, qml.PauliZ]:
                coeffs.append(coeff)
                ops.append(op(i) @ op(j))

    return qml.Hamiltonian(coeffs, ops)

# build sparse hamiltonian
H_list = [build_hamiltonian(J) for J in J_list]
H_sparse_list = [qml.utils.sparse_hamiltonian(H) for H in H_list]

# diagonalize
eigvals_list, eigvecs_list = [], []
ground_states_list = []
for H_sparse in tqdm(H_sparse_list):
    eigvals, eigvecs = sp.sparse.linalg.eigs(H_sparse, which='SR', k=1)
    eigvals = eigvals.real
    ground_state = eigvecs[:, np.argmin(eigvals)]
    eigvals_list.append(eigvals)
    eigvecs_list.append(eigvecs)
    ground_states_list.append(ground_state)

/home_new/tangyehui/miniconda3/envs/gqs/lib/python3.9/site-packages/pennylane/utils.py:67: UserWarning: The method sparse_hamiltonian is deprecated. Please use the method sparse_matrix of the Hamiltonian operator instead.
  warnings.warn(
100%|██████████| 20/20 [00:00<00:00, 30.77it/s]


In [388]:
# this circuit measures observables for the provided ground state
@qml.qnode(device=qml.device('default.qubit', wires=wires, shots=None))
def circ(observables):
    qml.QubitStateVector(ground_state, wires=range(wires))
    return [qml.expval(o) for o in observables]


def compute_exact_correlation_matrix(ground_state, wires):
    # setup observables for correlation function
    def corr_function(i, j):
        ops = []
        
        for op in [qml.PauliX, qml.PauliY, qml.PauliZ]:
            if i != j:
                ops.append(op(i) @ op(j))
            else:
                ops.append(qml.Identity(i))

        return ops
    
    # indices for sites for which correlations will be computed
    coupling_pairs = list(it.product(range(wires), repeat=2))
    
    # compute exact correlation matrix
    correlation_matrix = np.zeros((wires, wires))
    for idx, (i, j) in tqdm(enumerate(coupling_pairs), total=len(coupling_pairs)):
        observable = corr_function(i, j)

        if i == j:
            correlation_matrix[i][j] = 1.0
        else:
            correlation_matrix[i][j] = (
                    np.sum(np.array([circ(observables=[o]) for o in observable]).T) / 3
            )
            correlation_matrix[j][i] = correlation_matrix[i][j]

    return correlation_matrix

exact_correlation_matrix_list = [compute_exact_correlation_matrix(ground_state, wires) for ground_state in ground_states_list]

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 150.47it/s]


In [389]:
def compute_exact_entropy_matrix(ground_state, wires):
    ground_state_qobj = qutip.Qobj(ground_state, dims=[[2] * wires, [1] * wires])

    # compute entropies
    entropies = np.zeros(shape=(wires, wires), dtype=float)
    for i in tqdm(range(wires)):
        ptrace_diag = ground_state_qobj.ptrace(sel=[i])
        entropies[i, i] = -np.log(np.trace(ptrace_diag * ptrace_diag).real)

        for j in range(i + 1, wires):
            ptrace = ground_state_qobj.ptrace(sel=[i, j])
            e = -np.log(np.trace(ptrace * ptrace).real)
            entropies[i, j] = entropies[j, i] = e

    return entropies
exact_entropy_matrix_list = [compute_exact_entropy_matrix(ground_state, wires) for ground_state in ground_states_list]

100%|██████████| 10/10 [00:00<00:00, 430.60it/s]


In [390]:
from jax import jit
import jax.numpy as jnp

@jit
def _jax_compute_size_one_entropies(x):
    return -jnp.log(jnp.einsum('timl,silm->i', x, x))

@jit
def _jax_compute_size_two_entropies(x):
    return -jnp.log(jnp.einsum('tilm,siml,tjrk,sjkr->ij', x, x, x, x))

def compute_entropies_from_shadow(shadow):
    """
    compute second-order Rényi entanglement entropies for all subsystems of size at most two, using the classical shadow
    protocol
    """
    local_snapshots = shadow.local_snapshots()
    shadow_size = shadow.snapshots

    # compute size two entropies
    entropies = np.array(_jax_compute_size_two_entropies(local_snapshots) + 2 * np.log(shadow_size))

    # compute size one entropies
    entropies_size_one = np.array(_jax_compute_size_one_entropies(local_snapshots) + 2 * np.log(shadow_size))
    np.fill_diagonal(entropies, entropies_size_one)

    return entropies.real

def compute_correlation_matrix_from_shadow(shadow):
    wires = shadow.bits.shape[1]

    qubit_pairs = list(it.combinations(range(wires), r=2))

    correlations = np.zeros((wires, wires))
    np.fill_diagonal(correlations, 1.0)

    for idx, (i, j) in enumerate(qubit_pairs):
        obs = qml.PauliX(i) @ qml.PauliX(j) + qml.PauliY(i) @ qml.PauliY(j) + qml.PauliZ(i) @ qml.PauliZ(j)
        correlations[i, j] = correlations[j, i] = shadow.expval(H=obs, k=1) / 3

    return correlations

In [421]:
shots = 2**15
_samples = samples[:, :shots, :]
model_recipes_list = _samples // 2
model_bits_list = _samples - 2 * model_recipes_list

shadow_list = []
for i in tqdm(range(len(model_recipes_list))):
    shadow_list.append(qml.ClassicalShadow(bits=model_bits_list[i], recipes=model_recipes_list[i]))

100%|██████████| 20/20 [00:00<00:00, 20671.78it/s]


In [422]:
predicted_correlation_matrix_list = [compute_correlation_matrix_from_shadow(shadow=shadow_list[i]) for i in tqdm(range(len(shadow_list)))]

100%|██████████| 20/20 [01:01<00:00,  3.07s/it]


In [423]:
correlation_error = []
for i in range(len(predicted_correlation_matrix_list)):
    correlation_error.append((predicted_correlation_matrix_list[i] - exact_correlation_matrix_list[i])**2)
correlation_error = np.array(correlation_error)
correlation_error = np.mean(correlation_error)
correlation_error = np.sqrt(correlation_error)
correlation_error

0.19487802111329963